In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()

(5897, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,6.511986,1.881337,1.978520,0.048591,0.048591,1.8857,0.067659
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,6.0,1.9556,0.0886,6.992522,1.765439,1.961637,0.098099,0.098099,1.9556,0.088600
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,6.569075,1.798748,2.018901,0.110076,0.110076,1.7824,0.110076
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,6.0,1.8027,0.0586,6.290935,1.791712,1.973971,0.091129,0.091129,1.8027,0.067659
4,4,3,MS Parma 1,MS Parma,FIN,1,6.0,anastasia rudnaya,1,Anastasia Rudnaya,4.0,1.8899,0.0639,6.891132,1.875490,2.002510,0.063510,0.063510,1.8899,0.067659


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.067659
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9556,0.088600
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.110076
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.8027,0.067659
4,3,MS Parma 1,1,6.0,Anastasia Rudnaya,1.8899,0.067659


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                            \
leg                        1                    2                    3   
team_id                                                                  
1            Stora Tuna OK 1      Stora Tuna OK 1      Stora Tuna OK 1   
3                 MS Parma 1           MS Parma 1           MS Parma 1   
4        Järla Orientering 1  Järla Orientering 1  Järla Orientering 1   
5             Alfta ÖSA OK 1       Alfta ÖSA OK 1       Alfta ÖSA OK 1   
6                Ol Norska 1          Ol Norska 1          Ol Norska 1   

                             dist                               name  \
leg                        4    1    2    3    4                   1   
team_id                                                                
1            Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
3                 MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4        Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5             Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   
6                Ol Norska 1  6.0  5.7  7.3  7.9         Anja Probst   

                                                                       \
leg                        2                    3                   4   
team_id                                                                 
1           Magdalena Olsson          Julia Jakob  Tove Alexandersson   
3             Anni Heikkonen       Tiia Istolahti       Maija Sianoja   
4        Anna Dahlgren-Rosén  Elin Hemmyr-Skantze      Sofie Bachmann   
5            Josefine Heikka         Irina Nyberg    Natalia Gemperle   
6          Valérie Aebischer         Siri Nyfeler       Sarina Jenzer   

        log_mean                            log_std                      \
leg            1        2       3       4         1         2         3   
team_id                                                                   
1         1.8857  1.95560  1.7824  1.8027  0.067659  0.088600  0.110076   
3         1.8899  1.94010  1.8669  1.9157  0.067659  0.108600  0.128204   
4         1.9748  1.94591  1.8086  1.9289  0.112800  0.182322  0.124320   
5         1.8949  1.88830  1.9000  1.8720  0.110100  0.117383  0.095300   
6         2.0591  1.94591  2.0380  1.9768  0.089078  0.182322  0.153484   

                   
leg             4  
team_id            
1        0.067659  
3        0.067659  
4        0.087344  
5        0.067659  
6        0.135800

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1482, 20)

(1460, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,34.679601,71.731067,111.538771,157.768872,45.112489,89.351217,136.948818,186.651193,39.594344,79.960370,...,34.894534,42.034653,45.112489,47.950904,53.663356,54.756157,39.594344,40.279941,43.316297,47.903879
3,34.790137,70.297207,112.780691,164.229366,45.310084,90.125185,143.558255,198.715629,39.720884,79.437202,...,36.904275,47.039486,45.310084,49.006309,60.787094,61.075625,39.720884,39.532431,47.155037,53.687553
4,34.905842,68.417451,109.819237,162.360357,53.981793,103.046363,150.743750,206.715552,43.203411,83.496105,...,34.845042,45.709194,53.981793,56.841261,56.766618,64.683346,43.203411,39.907867,44.533631,54.334263
5,32.095233,66.605058,112.494876,162.218753,49.508105,90.958528,143.248519,196.274236,39.896574,77.858077,...,40.648673,45.071394,49.508105,47.609229,58.804055,58.615349,39.896574,37.757051,48.843400,51.286672
6,39.537310,72.482003,122.240233,176.239442,56.003751,105.992682,170.357274,232.632869,46.968304,87.320996,...,41.460664,43.594510,56.003751,57.105720,76.331628,74.996672,46.968304,39.918173,56.271698,57.158283


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
team_id,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,Julia Jakob,Tove Alexandersson,1.89,1.96,1.78,1.80,0.07,0.09,0.11,0.07
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,Tiia Istolahti,Maija Sianoja,1.89,1.94,1.87,1.92,0.07,0.11,0.13,0.07
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,Elin Hemmyr-Skantze,Sofie Bachmann,1.97,1.95,1.81,1.93,0.11,0.18,0.12,0.09
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,Irina Nyberg,Natalia Gemperle,1.89,1.89,1.90,1.87,0.11,0.12,0.10,0.07
6,Ol Norska 1,Ol Norska 1,Ol Norska 1,Ol Norska 1,6.0,5.7,7.3,7.9,Anja Probst,Valérie Aebischer,Siri Nyfeler,Sarina Jenzer,2.06,1.95,2.04,1.98,0.09,0.18,0.15,0.14


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,34.894534,42.034653,45.112489,47.950904,53.663356,54.756157,39.594344,40.279941,43.316297,47.903879
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,36.904275,47.039486,45.310084,49.006309,60.787094,61.075625,39.720884,39.532431,47.155037,53.687553
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.845042,45.709194,53.981793,56.841261,56.766618,64.683346,43.203411,39.907867,44.533631,54.334263
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.648673,45.071394,49.508105,47.609229,58.804055,58.615349,39.896574,37.757051,48.843400,51.286672
6,Ol Norska 1,Ol Norska 1,Ol Norska 1,Ol Norska 1,6.0,5.7,7.3,7.9,Anja Probst,Valérie Aebischer,...,41.460664,43.594510,56.003751,57.105720,76.331628,74.996672,46.968304,39.918173,56.271698,57.158283


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean   log_std  \
team_id leg                                                                  
1       1    Stora Tuna OK 1   6.0       Tilda Östberg    1.8857  0.067659   
        2    Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556  0.088600   
        3    Stora Tuna OK 1   7.3         Julia Jakob    1.7824  0.110076   
        4    Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027  0.067659   
3       1         MS Parma 1   6.0   Anastasia Rudnaya    1.8899  0.067659   
        2         MS Parma 1   5.7      Anni Heikkonen    1.9401  0.108600   
        3         MS Parma 1   7.3      Tiia Istolahti    1.8669  0.128204   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
1       1      34.679601   45.112489   39.594344   39.672176   
        2      71.731067   89.351217   79.960370   80.105666   
        3     111.538771  136.948818  123.443978  123.680156   
        4     157.768872  186.651193  171.545424  171.719011   
3       1      34.790137   45.310084   39.720884   39.809801   
        2      70.297207   90.125185   79.437202   79.635028   
        3     112.780691  143.558255  126.912433  127.227394   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.678406         0.066986         0.0              0.0   
        2            4.381789         0.055805         0.0              0.0   
        3            4.816304         0.052794         0.0              0.0   
        4            5.144954         0.042563         0.0              0.0   
3       1            3.681845         0.067362         0.0              0.0   
        2            4.375400         0.064069         0.0              0.0   
        3            4.844049         0.062055         0.0              0.0   

             personal_start95  personal_end95  personal_median  
team_id leg                                                     
1       1           34.679601       45.112489        39.594344  
        2           33.921352       47.950904        40.279941  
        3           34.894534       53.663356        43.316297  
        4           42.034653       54.756157        47.903879  
3       1           34.790137       45.310084        39.720884  
        2           32.080973       49.006309        39.532431  
        3           36.904275       60.787094        47.155037

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean   log_std  \
team_id leg                                                                  
1       1    Stora Tuna OK 1   6.0       Tilda Östberg    1.8857  0.067659   
        2    Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556  0.088600   
        3    Stora Tuna OK 1   7.3         Julia Jakob    1.7824  0.110076   
        4    Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027  0.067659   
3       1         MS Parma 1   6.0   Anastasia Rudnaya    1.8899  0.067659   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
1       1      34.679601   45.112489   39.594344   39.672176   
        2      71.731067   89.351217   79.960370   80.105666   
        3     111.538771  136.948818  123.443978  123.680156   
        4     157.768872  186.651193  171.545424  171.719011   
3       1      34.790137   45.310084   39.720884   39.809801   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.678406         0.066986         0.0              0.0   
        2            4.381789         0.055805         0.0              0.0   
        3            4.816304         0.052794         0.0              0.0   
        4            5.144954         0.042563         0.0              0.0   
3       1            3.681845         0.067362         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           34.679601       45.112489        39.594344   
        2           33.921352       47.950904        40.279941   
        3           34.894534       53.663356        43.316297   
        4           42.034653       54.756157        47.903879   
3       1           34.790137       45.310084        39.720884   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:35.660636902 2019-06-15 14:34:40.776042461   
        2   2019-06-15 15:19:57.622189283 2019-06-15 15:11:43.863991976   
        3   2019-06-15 16:03:26.638704538 2019-06-15 15:51:32.326252699   
        4   2019-06-15 16:51:32.725452423 2019-06-15 16:37:46.132331609   
3       1   2019-06-15 14:39:43.253029823 2019-06-15 14:34:47.408236027   

                           fin_time_end95  personal_estimate_interval  
team_id leg                                                            
1       1   2019-06-15 14:45:06.749319792                   10.432888  
        2   2019-06-15 15:29:21.073050022                   14.029552  
        3   2019-06-15 16:16:56.929096699                   18.768822  
        4   2019-06-15 17:06:39.071569920                   12.721504  
3       1   2019-06-15 14:45:18.605017900                   10.519946

In [21]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Jyry")]

for_print = for_print[[
 'leg',
 'name', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,
341,1,Jola Maisala,42.08,54.09,69.77,27.69
341,2,Kaija Rantoja,45.83,58.53,75.27,29.44
341,3,Jenni Castren,57.32,75.26,99.17,41.85
341,4,Inka Lemmetyinen,55.40,78.11,110.92,55.52
528,1,Päivi Taponen,54.50,63.51,74.41,19.90
528,2,Miia Hovi,42.31,51.31,62.25,19.93
528,3,Mervi Luurila,59.90,86.76,125.85,65.95
528,4,Tiina Isoniemi,59.13,68.15,78.69,19.55
740,1,Mila Viksilä,55.85,79.25,113.66,57.80


In [22]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1016,341,1,Rasti-Jyry 1,Jola Maisala,2.200700,0.130900,3.992258,0.129620,2019-06-15 11:42:05.012903929+00:00,2019-06-15 12:09:46.411721230+00:00,2019-06-15 11:54:05.347295761+00:00,0.0000,0.0
1017,341,2,Rasti-Jyry 1,Kaija Rantoja,2.328600,0.130600,4.729365,0.090868,2019-06-15 12:34:57.254395961+00:00,2019-06-15 13:15:08.037465096+00:00,2019-06-15 12:53:14.209855080+00:00,0.0000,0.0
1018,341,3,Rasti-Jyry 1,Jenni Castren,2.334200,0.141700,5.242832,0.078679,2019-06-15 13:42:08.035793543+00:00,2019-06-15 14:41:14.003121138+00:00,2019-06-15 14:09:05.211633444+00:00,0.0000,0.0
1019,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.293200,0.178309,5.592857,0.076915,2019-06-15 14:51:45.484045506+00:00,2019-06-15 16:11:57.095623970+00:00,2019-06-15 15:28:16.556379318+00:00,0.0000,0.0
1696,528,1,Rasti-Jyry 2,Päivi Taponen,2.361600,0.078900,4.152031,0.078878,2019-06-15 11:54:30.266949415+00:00,2019-06-15 12:14:24.494459391+00:00,2019-06-15 12:03:30.338958502+00:00,0.0000,0.0
1697,528,2,Rasti-Jyry 2,Miia Hovi,2.200400,0.099400,4.745775,0.062355,2019-06-15 12:42:00.302498341+00:00,2019-06-15 13:10:14.893716812+00:00,2019-06-15 12:55:03.472401619+00:00,0.0000,0.0
1698,528,3,Rasti-Jyry 2,Mervi Luurila,2.473200,0.191317,5.312116,0.090295,2019-06-15 13:51:01.954330444+00:00,2019-06-15 15:03:35.736713171+00:00,2019-06-15 14:22:33.918827772+00:00,0.0000,0.0
1699,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.155400,0.073500,5.603116,0.070304,2019-06-15 14:57:47.762012005+00:00,2019-06-15 16:12:29.622804165+00:00,2019-06-15 15:30:52.089706421+00:00,0.0017,0.0
2448,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.182322,4.372568,0.181329,2019-06-15 11:55:51.292377710+00:00,2019-06-15 12:53:39.377027988+00:00,2019-06-15 12:19:15.282169104+00:00,0.0000,0.0
2449,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.688600,0.138400,5.100805,0.113866,2019-06-15 13:11:56.388790846+00:00,2019-06-15 14:25:55.740413427+00:00,2019-06-15 13:44:04.074403763+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(5840, 13)

In [20]:
np.exp([-0.075241, 0.0, 0.01, 0.5])

array([0.92751993, 1.        , 1.01005017, 1.64872127])